In [4]:
import os
import time
import requests

API_TOKEN = ""
CARS_FOLDER = "cars"

def recognize_plate(image_path):
    """Call the plate recognition API with a single image and return the JSON."""
    url = "https://api.platerecognizer.com/v1/plate-reader/"
    with open(image_path, 'rb') as img_file:
        response = requests.post(
            url,
            files={'upload': img_file},
            headers={'Authorization': f'Token {API_TOKEN}'},
            # data={'regions': 'se'}  # If you specifically want region='se'
        )
    return response.json()

def detect_and_save_plates():
    """
    Walk through all subdirs in 'cars/'.
    If 'regnum.txt' does not already exist in a subfolder:
      1) Loop over all images (jpg, jpeg, png).
      2) Send each to the plate recognition API.
      3) As soon as we find a plate, save it to 'regnum.txt'
         and break out of the images loop (skip the rest).
    """
    for folder_name in os.listdir(CARS_FOLDER):
        folder_path = os.path.join(CARS_FOLDER, folder_name)
        
        # Skip if it's not a folder
        if not os.path.isdir(folder_path):
            continue
        
        regnum_file = os.path.join(folder_path, "regnum.txt")
        
        # If regnum.txt already exists, skip the folder
        if os.path.exists(regnum_file):
            print(f"[SKIP] Already found plate for '{folder_name}'.")
            continue

        # Gather all image files
        images = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        
        found_plate = False
        
        for img_name in images:
            img_path = os.path.join(folder_path, img_name)
            
            # Call the plate recognition API
            result = recognize_plate(img_path)
            
            if 'results' in result and len(result['results']) > 0:
                # For simplicity, just take the first recognized plate
                plate_obj = result['results'][0]
                plate_str = plate_obj['plate'].upper()
                
                # Write the plate to regnum.txt
                with open(regnum_file, "w", encoding="utf-8") as f:
                    f.write(plate_str)
                
                print(f"[FOUND] Plate '{plate_str}' in '{img_name}'.")
                
                found_plate = True
                break  # Stop analyzing more images in this folder
            
            # No plate found in this image, wait a sec to avoid throttling
            time.sleep(1)
        
        if not found_plate:
            print(f"[NO PLATE] No plate detected in any image under '{folder_name}'.")

def main():
    detect_and_save_plates()

if __name__ == "__main__":
    main()

[SKIP] Already found plate for 'Volkswagen'.
[SKIP] Already found plate for 'Porsche 992,911 Carrera 4 GTS Cabriolet PDK, garanti 2026-03'.
[SKIP] Already found plate for 'Volvo XC60 D4 Geartronic Classic, Momentum Euro 6'.
[SKIP] Already found plate for 'Ford Kuga 2.0 TDCi AWD Powershift Trend Euro 6'.
[SKIP] Already found plate for 'Audi A3 Sportback 1.0 TFSI Comfort Euro 6'.
[SKIP] Already found plate for 'Citroën Berlingo Multispace 1.6 BlueHDi 100 8v ETG6 Euro 6'.
[SKIP] Already found plate for 'Kia ceed jätte fint skick full service'.
[SKIP] Already found plate for 'Audi A4 allroad quattro 2.0 TDI DPF quattro Sport Edition Eu'.
[SKIP] Already found plate for 'Skoda Superb Kombi 2.0 TDI 4x4 Elegance Euro 5'.
[SKIP] Already found plate for 'Fiat Abarth 595 1.4 T-JET 16V Turismo Euro 6'.
[SKIP] Already found plate for 'Volvo V90 Recharge T8 AWD Geartronic R-Design Euro6 Polestar'.
[SKIP] Already found plate for 'BMW X1 xDrive20d Sport line Euro 5'.
[SKIP] Already found plate for 'Te

In [5]:
import os
import shutil

CARS_FOLDER = "cars"

def clean_folders_without_plate():
    for folder_name in os.listdir(CARS_FOLDER):
        folder_path = os.path.join(CARS_FOLDER, folder_name)
        
        if not os.path.isdir(folder_path):
            continue

        regnum_file = os.path.join(folder_path, "regnum.txt")
        
        if not os.path.exists(regnum_file):
            print(f"[DELETE] '{folder_name}' does not contain 'regnum.txt', deleting folder.")
            shutil.rmtree(folder_path)
        else:
            print(f"[KEEP] '{folder_name}' contains 'regnum.txt'.")

if __name__ == "__main__":
    clean_folders_without_plate()


[KEEP] 'Volkswagen' contains 'regnum.txt'.
[KEEP] 'Porsche 992,911 Carrera 4 GTS Cabriolet PDK, garanti 2026-03' contains 'regnum.txt'.
[KEEP] 'Volvo XC60 D4 Geartronic Classic, Momentum Euro 6' contains 'regnum.txt'.
[KEEP] 'Ford Kuga 2.0 TDCi AWD Powershift Trend Euro 6' contains 'regnum.txt'.
[KEEP] 'Audi A3 Sportback 1.0 TFSI Comfort Euro 6' contains 'regnum.txt'.
[KEEP] 'Citroën Berlingo Multispace 1.6 BlueHDi 100 8v ETG6 Euro 6' contains 'regnum.txt'.
[KEEP] 'Kia ceed jätte fint skick full service' contains 'regnum.txt'.
[KEEP] 'Audi A4 allroad quattro 2.0 TDI DPF quattro Sport Edition Eu' contains 'regnum.txt'.
[KEEP] 'Skoda Superb Kombi 2.0 TDI 4x4 Elegance Euro 5' contains 'regnum.txt'.
[KEEP] 'Fiat Abarth 595 1.4 T-JET 16V Turismo Euro 6' contains 'regnum.txt'.
[KEEP] 'Volvo V90 Recharge T8 AWD Geartronic R-Design Euro6 Polestar' contains 'regnum.txt'.
[KEEP] 'BMW X1 xDrive20d Sport line Euro 5' contains 'regnum.txt'.
[KEEP] 'Tesla Model 3 Long Range AWD_Autopilot, Dator för 

In [2]:
import os
import shutil

CARS_FOLDER = "cars"

def is_valid_plate(plate):
    """Returns True if plate is exactly 6 alphanumeric characters and contains both letters and digits."""
    return (
        plate 
        and len(plate.strip()) == 6 
        and plate.strip().isalnum() 
        and any(char.isdigit() for char in plate) 
        and any(char.isalpha() for char in plate)
    )

def clean_invalid_plates():
    """Delete car folders with missing, incorrectly sized, or invalid regnum.txt plates."""
    deleted = 0
    checked = 0

    for folder_name in os.listdir(CARS_FOLDER):
        folder_path = os.path.join(CARS_FOLDER, folder_name)
        if not os.path.isdir(folder_path):
            continue

        reg_file = os.path.join(folder_path, "regnum.txt")
        checked += 1

        if not os.path.exists(reg_file):
            print(f"[DELETE] No regnum.txt → {folder_name}")
            shutil.rmtree(folder_path)
            deleted += 1
            continue

        with open(reg_file, "r", encoding="utf-8") as f:
            plate = f.read().strip()

        if not is_valid_plate(plate):
            print(f"[DELETE] Invalid plate '{plate}' → {folder_name}")
            shutil.rmtree(folder_path)
            deleted += 1
        else:
            print(f"[KEEP] Valid plate '{plate}' in {folder_name}")

    print(f"\n[SUMMARY] Checked: {checked} folders, Deleted: {deleted}")

if __name__ == "__main__":
    clean_invalid_plates()


[KEEP] Valid plate 'TNG159' in Volkswagen
[DELETE] Invalid plate 'GJD48X' → Ford Kuga 2.0 TDCi AWD Powershift Trend Euro 6
[KEEP] Valid plate 'LRO335' in Audi A3 Sportback 1.0 TFSI Comfort Euro 6
[KEEP] Valid plate 'ONS641' in Citroën Berlingo Multispace 1.6 BlueHDi 100 8v ETG6 Euro 6
[KEEP] Valid plate 'TYX878' in Skoda Superb Kombi 2.0 TDI 4x4 Elegance Euro 5
[DELETE] Invalid plate 'HSD80F' → Volvo V90 Recharge T8 AWD Geartronic R-Design Euro6 Polestar
[KEEP] Valid plate 'DXZ218' in BMW X1 xDrive20d Sport line Euro 5
[DELETE] Invalid plate 'HHD98C' → Tesla Model 3 Long Range AWD_Autopilot, Dator för FSD
[KEEP] Valid plate 'RNP392' in BMW 116 i 5-dörrars M Sport Euro 6
[KEEP] Valid plate 'YXG040' in BMW 318 d Sedan Advantage Euro 6
[KEEP] Valid plate 'PWL471' in Opel Astra 1.4 Turbo Enjoy Euro 5
[KEEP] Valid plate 'YZJ303' in Audi Q7 3.0 TDI V6 quattro TipTronic S Line
[KEEP] Valid plate 'RMF105' in Renault Clio 1.2 Expression Euro 6
[KEEP] Valid plate 'LAN373' in Mercedes-Benz E 220 